# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [791]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [792]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [793]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [794]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [795]:
players_180_190 = int(player_data.loc[(player_data.height >= 180) & \
                                  (player_data.height <= 190),'id'].count())
players_180_190

5925

In [796]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [797]:
from datetime import datetime
players_1980 = int(player_data.loc[(pd.to_datetime(player_data.birthday) >= datetime(1980,1,1)) & \
                                   (pd.to_datetime(player_data.birthday) < datetime(1981,1,1)), 'id'].count())
players_1980

434

In [798]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [799]:
highest_players = list(player_data.sort_values(by=['weight','player_name'], ascending=[False,True]).iloc[:10].player_name)
highest_players

['Kristof van Hout',
 'Tim Wiese',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Cassio',
 'Christopher Samba',
 'Lars Unnerstall',
 'Marcus Hahnemann',
 'Abdoulaye Faye',
 'Enoch Showunmi']

In [800]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [801]:
def birthday_to_year(row):
    row.birthday = int((pd.to_datetime(row.birthday)).year)
    return row

name_year = player_data.loc[:, ['player_name', 'birthday']].copy()
name_year.apply(birthday_to_year, axis='columns')
name_year = name_year.rename(columns = {'birthday': 'year', 'player_name':'name'})
name_year = name_year.loc[(name_year.year >= 1980) & (name_year.year <= 1990)]
stats = name_year.groupby('year').year.count()

years_born_players = list(zip(stats.index,stats))
years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

In [802]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [803]:
name_height = player_data.loc[:, ['player_name','height']].copy()
name_height.player_name = name_height.player_name.apply(lambda n : n.split()[0])
adriano_mean = name_height.loc[name_height.player_name == 'Adriano'].height.mean()
adriano_std = name_height.loc[name_height.player_name == 'Adriano'].height.std()
adriano_mean,adriano_std

(182.11800000000002, 5.361479480723788)

In [804]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [805]:
name_datetime = player_data.loc[:, ['player_name','birthday']].copy()
name_datetime['birthday'] = pd.to_datetime(name_datetime['birthday'])
name_datetime['birthday'] = name_datetime['birthday'].dt.day_name()
name_datetime = name_datetime.rename(columns = {'birthday':'day'})

dow_with_min_players_born = name_datetime.groupby('day').day.count().sort_values().index[0]
dow_with_min_players_born

'Sunday'

In [806]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [807]:
match_data = pd.read_sql("SELECT * FROM Match;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)

left = match_data.loc[:, ['id','league_id']]
left = left.set_index('league_id')

right = league_data.loc[:, ['id','name']]
right = right.set_index('id')

match_league_data = left.join(right)

stats = match_league_data.groupby('name').id.count().sort_values(ascending=False)
league_most_matches = stats.loc[stats >= stats[0]].sort_index().index[0]
league_most_matches

'England Premier League'

In [808]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [858]:
key_field = 'player_api_id';
player_names = player_data.loc[:, [key_field, 'player_name']]
player_names[key_field] = player_names[key_field].astype('int64')
player_names = player_names.set_index(key_field);


def to_player_data(match_data, field, player_names):
    h1_data = match_data[pd.notnull(match_data[str(field)])].loc[:, [str(field), 'match_api_id']].copy();
    h1_data[str(field)] = h1_data[str(field)].astype('int64')
    h1_data = h1_data.set_index(str(field))
    joined = h1_data.join(player_names)
    return joined

merged = pd.concat([
    to_player_data(match_data, 'home_player_1', player_names),
    to_player_data(match_data, 'home_player_2', player_names),
    to_player_data(match_data, 'home_player_3', player_names),
    to_player_data(match_data, 'home_player_4', player_names),
    to_player_data(match_data, 'home_player_5', player_names),
    to_player_data(match_data, 'home_player_6', player_names),
    to_player_data(match_data, 'home_player_7', player_names),
    to_player_data(match_data, 'home_player_8', player_names),
    to_player_data(match_data, 'home_player_9', player_names),
    to_player_data(match_data, 'home_player_10', player_names),
    to_player_data(match_data, 'home_player_11', player_names),
    to_player_data(match_data, 'away_player_1', player_names),
    to_player_data(match_data, 'away_player_2', player_names),
    to_player_data(match_data, 'away_player_3', player_names),
    to_player_data(match_data, 'away_player_4', player_names),
    to_player_data(match_data, 'away_player_5', player_names),
    to_player_data(match_data, 'away_player_6', player_names),
    to_player_data(match_data, 'away_player_7', player_names),
    to_player_data(match_data, 'away_player_8', player_names),
    to_player_data(match_data, 'away_player_9', player_names),
    to_player_data(match_data, 'away_player_10', player_names),
    to_player_data(match_data, 'away_player_11', player_names),
]).drop_duplicates()

player_id = merged.index.value_counts().index[0]

max_matches_player = list(player_data.loc[player_data.player_api_id==player_id].player_name)[0]
max_matches_player

'Steve Mandanda'

In [810]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [811]:
attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
columns = [column for column in attributes_data.columns if column not in ['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']]
att_data = attributes_data.loc[:, columns]
corr_matrix = att_data.corr(method='pearson').abs()

stats_list = corr_matrix.unstack()
pairs = list(stats_list.sort_values(kind="quicksort", ascending=False).index)


def filter_pairs(pairs):
    unique_pairs = [pair for pair in pairs if pair[0] != pair[1]]

    CNT_MAX = 5
    top_pairs = []
    top_reversed_pairs = []
    for pair in unique_pairs:
        if len(top_pairs) == 0:
            top_pairs = [pair]
            top_reversed_pairs = [(pair[1], pair[0])]

        if (pair not in top_pairs) and (pair not in top_reversed_pairs):
            top_pairs.append(pair)
            top_reversed_pairs.append((pair[1], pair[0]))

        if len(top_pairs) == CNT_MAX:
            return top_pairs

    return top_pairs


top_correlated_features = filter_pairs(pairs)
top_correlated_features

[('gk_reflexes', 'gk_positioning'),
 ('gk_reflexes', 'gk_handling'),
 ('gk_handling', 'gk_positioning'),
 ('sliding_tackle', 'standing_tackle'),
 ('standing_tackle', 'marking')]

In [812]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [813]:
attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)

import math
def distance (list1, list2):
    return math.sqrt(sum([(s - m)**2 for s, m in zip(list1, list2)]))

player_attributes_data = attributes_data.groupby('player_api_id').date.agg([max])
player_attributes_data = player_attributes_data.rename(columns={'max':'max_date'})

attributes_last_data = attributes_data.copy()
attributes_last_data = attributes_last_data.set_index('player_api_id')

last_attributes_data = attributes_last_data.join(player_attributes_data)
last_attributes_data = last_attributes_data[last_attributes_data.date == last_attributes_data.max_date]

columns = [column for column in attributes_data.columns if column not in ['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']]

attributes = last_attributes_data.loc[:, columns]

neymar_id = int(player_data[player_data.player_name == 'Neymar'].player_api_id)
neymar_attributes = attributes.loc[attributes.index == neymar_id]
neymar_list = neymar_attributes.values.tolist()[0]


attributes['dist'] = attributes.apply(lambda p: distance(p.values.tolist(), neymar_list) , axis='columns')

players = player_data.loc[:, ['player_api_id', 'player_name']]
players = players.set_index('player_api_id')

stats = attributes.join(players)
stats = stats.sort_values(by=['dist', 'player_name'], ascending=[True,True]).loc[:,['player_name','dist']]
stats = stats[stats.player_name != 'Neymar']
neymar_similarities = list(stats.iloc[:5].player_name)
neymar_similarities

# neymar_similarities = ['','','','','']

['Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

In [814]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [815]:
country_data = pd.read_sql("SELECT * FROM Country;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)

germany_id = int(country_data[country_data.name == 'Germany'].id)
team_id = int(team_data[team_data.team_long_name == 'Borussia Dortmund'].team_api_id)


home_matches = match_data.loc[(match_data.home_team_api_id == team_id) & \
                              (match_data.country_id == germany_id) & \
                              (match_data.season == '2008/2009')]

borussia_bundesliga_2008_2009_matches = int(home_matches.match_api_id.count())
borussia_bundesliga_2008_2009_matches

17

In [816]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [817]:
league_data = pd.read_sql("SELECT * FROM League;", db)

bundesliga_id = int(league_data[league_data.name == 'Germany 1. Bundesliga'].id)

bundesliga_matches = match_data.loc[(match_data.league_id == bundesliga_id) & \
                                    (match_data.season == '2008/2009'), 
                                   ['id' ,'home_team_api_id', 'away_team_api_id', ]]

home_matches = bundesliga_matches.loc[:, ['id', 'home_team_api_id']]
home_matches = home_matches.rename(columns={'home_team_api_id': 'team_api_id'})
home_matches.team_api_id = home_matches.team_api_id.astype('int64')

away_matches = bundesliga_matches.loc[:, ['id', 'away_team_api_id']]
away_matches = away_matches.rename(columns={'away_team_api_id': 'team_api_id'})
away_matches.team_api_id = away_matches.team_api_id.astype('int64')

matches = pd.concat([home_matches, away_matches])

stats = matches.groupby('team_api_id').team_api_id.count().sort_values(ascending=False)
team_most_matches_bundesliga_2008_2009 = int(stats.iloc[0])
team_most_matches_bundesliga_2008_2009

34

In [818]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [819]:
arsenal_id = int(team_data[team_data.team_long_name == 'Arsenal'].team_api_id)

home_matches = match_data.loc[(match_data.home_team_api_id == arsenal_id) & \
                                 (match_data.season == '2015/2016')]
away_matches = match_data.loc[(match_data.away_team_api_id == arsenal_id) & \
                                 (match_data.season == '2015/2016')]

won_home_matches = int(home_matches[home_matches.home_team_goal > home_matches.away_team_goal].id.count())
won_away_matches = int(away_matches[away_matches.home_team_goal < away_matches.away_team_goal].id.count())

arsenal_won_matches_2015_2016 = won_home_matches + won_away_matches
arsenal_won_matches_2015_2016

20

In [820]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [821]:
teams = team_data.loc[:, ['team_api_id', 'team_long_name']].set_index('team_api_id')

matches = match_data.loc[(match_data.season == '2015/2016'), ['match_api_id', 'home_team_api_id', 'home_team_goal', 'away_team_api_id', 'away_team_goal']]
matches.match_api_id = matches.match_api_id.astype('int64')

home_matches = matches.set_index('home_team_api_id').loc[:,['match_api_id', 'home_team_goal', 'away_team_goal']]
away_matches = matches.set_index('away_team_api_id').loc[:,['match_api_id', 'home_team_goal', 'away_team_goal']]

team_home_matches = home_matches.join(teams)
team_home_matches['success'] = team_home_matches.home_team_goal > team_home_matches.away_team_goal
team_home_matches.loc[(team_home_matches.index == arsenal_id) & \
                      (team_home_matches.success == True)].match_api_id.count()

team_away_matches = away_matches.join(teams)
team_away_matches['success'] = team_away_matches.home_team_goal < team_away_matches.away_team_goal
team_matches = pd.concat([team_home_matches, team_away_matches])

team_total_matches_s = team_matches.groupby('team_long_name').team_long_name.count().sort_values(ascending=False)
team_total_matches = pd.DataFrame({'name':team_total_matches_s.index, 'total':team_total_matches_s.values})
team_total_matches = team_total_matches.set_index('name')

team_won_matches_s = team_matches[team_matches.success == True].groupby('team_long_name').team_long_name.count().sort_values(ascending=False)
team_won_matches = pd.DataFrame({'name':team_won_matches_s.index, 'won':team_won_matches_s.values})
team_won_matches = team_won_matches.set_index('name')

result_table = team_won_matches.join(team_total_matches)
result_table['factor'] = result_table.won/result_table.total
result_table

stats = result_table.sort_values(by=['factor', 'name'], ascending=[False, True])
stats[stats.index=='Arsenal']

team_highest_winrate_2015_2016 = result_table.sort_values(by=['factor', 'name'], ascending=[False, True]).index[0]
team_highest_winrate_2015_2016

'SL Benfica'

In [822]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [829]:
teams = team_data.loc[:, ['team_api_id', 'team_long_name']].set_index('team_api_id')
league_id = int(league_data[league_data.name == 'England Premier League'].id)

matches = match_data.loc[(match_data.season == '2010/2011') & (match_data.league_id == league_id),
                          ['match_api_id', 'date', 'home_team_api_id', 'away_team_api_id']]
matches.match_api_id = matches.match_api_id.astype('int64')

home_matches = matches.set_index('home_team_api_id').loc[:,['match_api_id', 'date']]
away_matches = matches.set_index('away_team_api_id').loc[:,['match_api_id', 'date']]

team_home_matches = home_matches.join(teams)
team_away_matches = away_matches.join(teams)
team_matches = pd.concat([team_home_matches, team_away_matches])
team_matches.date = pd.to_datetime(team_matches.date)

league_teams = list(set(team_matches.team_long_name))

gaps = []
for team_name in league_teams:
    dates = list(team_matches.loc[team_matches.team_long_name == team_name, 'date'].sort_values())
    dates_prev = dates[:-2]
    dates_next = dates[1:]
    gap = [(n-p).days for p,n in zip(dates_prev,dates_next)]
    gaps.append(max(gap))
    
team_gap = pd.DataFrame({'team':league_teams, 'gap':gaps})
stats = team_gap.sort_values(by='gap', ascending=False)
highest_gap_england_2010_2011 = int(stats.gap.max())
highest_gap_england_2010_2011

18

In [826]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [827]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")